In [ ]:
!pip install sqlalchemy
!pip install pymysql
!pip show sqlalchemy | grep Version
!pip install --upgrade sqlalchemy

In [ ]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import getpass  # To get the password without showing the input
password = getpass.getpass()

# Note that when you use _SQLAlchemy_ and establish the connection, you do not even need to be logged in Sequel Pro or MySQL Workbench.

In [ ]:
bd = "sakila"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/'+bd
engine = create_engine(connection_string)
engine

In [ ]:
!pip install cryptography


In [ ]:
from sqlalchemy import text

with engine.connect() as connection:
    query = text('SELECT * FROM actor')
    result = connection.execute(query)
    for row in result.mappings():
        print(row)

Challenge
In this lab, the objective is to identify the customers who were active in both May and June, and how did their activity differ between months. To achieve this, follow these steps:

Establish a connection between Python and the Sakila database. X

Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.
month: an integer representing the month for which rental data is to be retrieved.
year: an integer representing the year for which rental data is to be retrieved.
The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".

Hint: Consider making use of pandas groupby()

Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.



In [ ]:
with engine.connect() as connection:
    query = text('SELECT * FROM rental')
    result = connection.execute(query)
    for row in result.mappings():
        print(row)

In [ ]:
# Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFram
def rentals_month(engine, month, year):
    query = f"""
    SELECT rental_id, rental_date, inventory_id, customer_id, return_date, staff_id, last_update
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year};
    """
    df = pd.read_sql(query, engine)
    df['rental_date'] = pd.to_datetime(df['rental_date'])
    df['return_date'] = pd.to_datetime(df['return_date'])
    df['last_update'] = pd.to_datetime(df['last_update'])
    return df

In [ ]:
rentals_may = rentals_month(engine, 5, 2005)
rentals_may

In [ ]:
rentals_august = rentals_month(engine, 8, 2005)
rentals_august
rentals_june = rentals_month(engine, 6, 2005)


In [ ]:
#Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along 
#with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during 
#the selected month and year.
def rental_count_month(rentals_df, month, year):
    rentals_count_df = rentals_df.groupby('customer_id').size().reset_index(name=f'rentals_in_{month:02}/{year}')
    return rentals_count_df


In [ ]:
rental_counts_may = rental_count_month(rentals_may, 5, 2005)
rental_counts_may


In [ ]:
rental_counts_august = rental_count_month(rentals_august, 8, 2005)
rental_counts_august
rental_counts_june = rental_count_month(rentals_june, 6, 2005)


In [ ]:
#Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals
#made by each customer in different months and years. The function should return a combined DataFrame with a new 
#'difference' column, which is the difference between the number of rentals in the two months.
def compare_rentals (df1, df2):
    combined_df = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    rental_month1 = df1.columns[1]
    rental_month2 = df2.columns[1]
    combined_df['difference'] = combined_df[rental_month1] - combined_df[rental_month2]
    # combined_df = combined_df.sort_values(by='difference', ascending=False)
    return combined_df
    

In [ ]:
compare_result = compare_rentals(rental_counts_may,rental_counts_august)

In [ ]:
compare_result

In [ ]:
compare_result2 = compare_rentals(rental_counts_may,rental_counts_june)
compare_result2